## Adult Dataset Part 1: Cleaning and Preparation

#### Objectives:
1. Preprocess the data and store the cleaned dataset as adult_clean.csv
2. Load clean dataset and test supervised/unsupervised models
    - Goal: Determine best model to predict if new entry earns >50k or <50k with supervised learning
    - Goal: Find insights and patterns in data using unspuervised learning
3. Apply k-folds cross validation
    - Goal: Evaluate robustness of results across different models.
    - Goal: Use and justify different error metrics


#### Summary of dataset

Datasource: https://archive.ics.uci.edu/ml/datasets/adult

Feature descrptions copied below from the UCI website:

- age: continuous. 
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
- fnlwgt: Continuous. A weighting assigned from the sampling process.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
- education-num: continuous. 
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
- sex: Female, Male. 
- capital-gain: continuous.
- capital-loss: continuous. 
- hours-per-week: continuous. 
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- target: >50K, <=50K.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#original data does not come with headers. add headers based on the documentation descrption
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 'target']

In [ ]:
data = pd.read_csv('./data/adult.data', names = headers)
df = data.copy()
df.head()

In [ ]:
df.info()

On first look, it appears there are no missing values. **However the dataset descrption indicates that missing values have been flled with '?'.** Figure out what to do here :)

Before deciding **how to deal with the missing values want to know how they influence the data**. For example, if workclass and occupation for example both have missing values (in the same tuple) it would be reasonable to drop the data.

In [ ]:
df = df.replace(['?'], 0)
df = df.drop('education-num', axis=1)
df = df.drop('fnlwgt', axis=1)
df.head()

In [ ]:
df.shape

**What % of the dataset do nans represent?** What is the **best approach** for dealing with the nan's?

In [ ]:
df2 = df.dropna(axis=0)
df.shape, df2.shape

# for i in df.columns:
#     x = df[i].isin([np.nan]).sum()
#     print(x)

# df.to_csv(r'C:\Users\Igor\Documents\GitHub\AI-Engineering\Chapter 2\07. SVM\data\adult_clean.csv')

# df.head()

# Load Clean Data

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix
from sklearn.svm import SVC, SVR
from sklearn.decomposition import PCA

In [ ]:
clean_data = pd.read_csv(r'C:\Users\Igor\Documents\GitHub\AI-Engineering\Chapter 2\07. SVM\data\adult_clean.csv')
clean_data = clean_data.drop('Unnamed: 0', axis=1)

clean_data_numerical = clean_data.copy()

In [ ]:
clean_data.head()

In [ ]:
y = df['target'].replace( {' >50K', ' <=50K'}, {1, 0} )

categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
numerical_cols = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']
sns.pairplot(clean_data[numerical_cols], hue=y)


In [ ]:
le = LabelEncoder()

for i in clean_data_numerical.columns:
    if clean_data_numerical[i].dtype == 'object':
        clean_data_numerical[i] = le.fit_transform(clean_data_numerical[i])

clean_data_numerical.head()


y = clean_data_numerical.iloc[:,-1]
sns.pairplot(clean_data_numerical, hue=y)

In [ ]:
x = clean_data_numerical.iloc[:,:-1]
y = clean_data_numerical.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=0)
scaler = StandardScaler()
mm_scaler = MinMaxScaler()

x_train_scaled = mm_scaler.fit_transform(x_train)
x_test_scaled = mm_scaler.transform(x_test)

clf = SVC()
clf.fit(x_train, y_train)

preds = clf.predict(x_test)
acc = accuracy_score(preds, y_test)
acc